In [38]:
import csv
class NFA:
    def __init__(self, name="N1"):
        self.name = name
        self.transitions = []
        self.state_counter = 0
        self.states = set()
        self.alphabet = set()
        self.start_state = self.new_state()
        self.final_states = set()

    def new_state(self):
        state = f"q{self.state_counter}"
        self.state_counter += 1
        self.states.add(state)
        return state

    def add_transition(self, start, input_symbol, end):
        self.transitions.append((start, input_symbol, end))
        if input_symbol != "∼":  # Epsilon transitions are not part of the alphabet
            self.alphabet.add(input_symbol)
    def build_nfa_from_regex(self, regex):
        start, end = self.parse_sub_regex(regex, 0, len(regex))
        self.final_states.add(end)
        return start, end

    def parse_sub_regex(self, regex, start_index, end_index):
        start_state = self.new_state()
        current_state = start_state

        index = start_index
        while index < end_index:
            char = regex[index]

            if char == '(':
                # Find the matching closing parenthesis
                brace_count, sub_end = 1, index + 1
                while brace_count > 0 and sub_end < end_index:
                    if regex[sub_end] == '(':
                        brace_count += 1
                    elif regex[sub_end] == ')':
                        brace_count -= 1
                    sub_end += 1

                # Recursively process the contents of the parentheses
                sub_start, sub_end_state = self.parse_sub_regex(regex, index + 1, sub_end - 1)
                self.add_transition(current_state, "∼", sub_start)
                current_state = sub_end_state
                index = sub_end  # Skip past the closing parenthesis

            elif char == '*':
                loop_start = self.new_state()
                loop_end = self.new_state()
                self.add_transition(current_state, "∼", loop_start)
                self.add_transition(loop_start, "∼", loop_end)
                self.add_transition(loop_end, "∼", loop_start)
                current_state = loop_end

            elif char == 'U':
                # Handle union operator
                next_start, next_end = self.parse_sub_regex(regex, index + 1, end_index)
                union_start = self.new_state()
                union_end = self.new_state()
                self.add_transition(union_start, "∼", start_state)
                self.add_transition(union_start, "∼", next_start)
                self.add_transition(current_state, "∼", union_end)
                self.add_transition(next_end, "∼", union_end)
                current_state = union_end
                break  # Union operation splits the processing flow

            else:
                next_state = self.new_state()
                self.add_transition(current_state, char, next_state)
                current_state = next_state

            index += 1

        return start_state, current_state

    def write_to_csv(self, filename_suffix='REGEX_TO_NFA'):
        output_filename = f"{filename_suffix}_{self.name}.csv"
        with open(output_filename, 'w', newline='') as file:
            writer = csv.writer(file)

            writer.writerow([f'{filename_suffix} {self.name},,,'])
            
            # States including accept states
            states_with_asterisk = {f"*{state}" if state in self.final_states else state for state in self.states}
            writer.writerow(sorted(states_with_asterisk) + [''] * (4 - len(states_with_asterisk)))

            # Alphabet
            writer.writerow(sorted(self.alphabet) + [''] * (3 - len(self.alphabet)))

            # Start state
            writer.writerow([self.start_state] + [''] * 3)

            # Accept states
            writer.writerow(sorted({f"*{state}" for state in self.final_states}) + [''] * (4 - len(self.final_states)))

            # Transitions
            for start, input_symbol, end in self.transitions:
                marked_start = f"*{start}" if start in self.final_states else start
                marked_end = f"*{end}" if end in self.final_states else end
                writer.writerow([marked_start, input_symbol, marked_end, ''])

        print(f"{filename_suffix} {self.name} saved to {output_filename}")



# Example Usage
nfa_converter = NFA()
nfa_converter.build_nfa_from_regex("a(abb)*Ub")
nfa_converter.write_to_csv("re-nfa.csv")


re-nfa.csv N1 saved to re-nfa.csv_N1.csv
